In [ ]:
import json, pickle, warnings
import pandas as pd
import numpy as np
from random import sample
from os.path import join
import seaborn as sns

from scipy.stats import ttest_ind

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, classification_report, precision_score, recall_score
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

from statsmodels.stats.contingency_tables import mcnemar

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

warnings.filterwarnings('ignore')

In [ ]:
from data_manipulations import *

## Data

In [ ]:
data_dir = join('data', '05_preprocessed')

In [ ]:
raw_data = pd.read_csv(join(data_dir, 'all_data_combined.csv'))
data = raw_data.copy()
data_gf = pd.read_csv(join(data_dir, 'all_data_combined_GF.csv'))

In [ ]:
raw_data['CT//StudyDate'].describe()

In [ ]:
drop_columns = ['Location', 'Behandlungsplan//Behandlungsprotokoll::Sauerstofftherapie']
data.drop(drop_columns, inplace=True, axis=1)
data_gf.drop(drop_columns, inplace=True, axis=1)

In [ ]:
# Hyperparameter optimization
perform_ho = True
n_iter = 10
cv = 3
verbose = 3
random_state = 42

In [ ]:
with open(join('data', 'features', 'selected_features.json'), 'r') as f:
    features = json.load(f)

## Visualization

In [ ]:
df = data_gf.copy()

In [ ]:
df['Target_3_DEC'] = df['Target_3'].map({0: 'Non-severe', 1: 'Severe'})

In [ ]:
target_variable = 'Target_3_DEC'
figure, axis = plt.subplots(1, 2, figsize=(10,6))
figure.tight_layout(pad=5.0)
plot_features = ['Number_comorbidities',
                 'CT//Konsolidierung::Anzahl betroffener Lappen'
                ]
y_labels = ['Number of comorbidities', 'Number of affected lung lobes (consolidation)']

for i, col in enumerate(plot_features):
    plot = sns.boxplot(data=df, x=target_variable, y=col, ax=axis[i], color='#c4c4c4', width=0.3, linewidth=0.8)
    plot.set_xlabel("Disease progression",fontsize=11, labelpad=10)
    plot.set_ylabel(y_labels[i], fontsize=11)

In [ ]:
comorbs = [col for col in df.columns if 'Komorbiditäten' in col]
for i, comorb in enumerate(comorbs):
    df[comorb] = df[comorb].replace(-1, 0)
    g = df.groupby(comorb)['Target_3_DEC'].value_counts(normalize=True).unstack()
    ax = g.plot(kind='bar', figsize=(7, 5), xlabel=comorb.split('::')[1].replace('##CODE', ''), ylabel='Percentage', rot=0, stacked=True)  # , ax=axe[i]
    ax.legend(title='Disease progression', bbox_to_anchor=(1, 1), loc='upper left')

## Risk model

**Target variables**

* **Target 0: Höchster Behandlungsstatus**

Unique values: 0 - Unbekannt 1 - Ambulant 2 - Notaufnahme 3 - Stationär 4 - IMC 5 - ICU

* **Target 2: Höchster Beatmungsbedarf**

Unique values: 0 - Unbekannt 1 - Nasenbrille 2 - Nicht-invasive Beatmung 3 - Invasive Beatmung 4 - Invasive Beatmung mit ECMO

* **Target 3: Schweregrad der COVID-19 Erkrankung**

Unique values: 1 - Schwer 0 - Leicht

### Feature subsets

In [ ]:
forbidden_features = ['PatientID', 'Target_0', 'Target_2', 'Target_3']
not_incl_features = [col for col in data.columns if col not in forbidden_features]

In [ ]:
drop_comorb_features = [
    'CT//Milchglasareal::Schweregrad (Mittellappen rechts)',
    'CT//Milchglasareal::Schweregrad (Oberlappen links)',
    'CT//Milchglasareal::Schweregrad (Oberlappen rechts)',
    'CT//Milchglasareal::Schweregrad (Unterlappen links)',
    'CT//Milchglasareal::Schweregrad (Unterlappen rechts)',
    'CT//Severity Scores::Lunge Mittelfeld links',
    'CT//Severity Scores::Lunge Mittelfeld rechts',
    'CT//Severity Scores::Lunge Oberfeld links',
    'CT//Severity Scores::Lunge Oberfeld rechts',
    'CT//Severity Scores::Lunge Unterfeld links',
    'CT//Severity Scores::Lunge Unterfeld rechts',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Emphysem',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Lungenfibrose',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Gibt es andere bekannte Komorbiditäten?',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Chronisch obstruktive Lungenerkrankung',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Bluthochdruck',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Herzerkrankungen',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Stauung/Ödem',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Dialyse',
    'Klinisch-anamnestische Information//Komorbiditäten aus Arztbrief::Diabetes mellitus##CODE'
]

In [ ]:
# Subset I: All features (data)
I_features = [col for col in data.columns if col not in forbidden_features]

# Subset II: All features + generated features (data_gf)
II_features = [col for col in data_gf.columns if col not in forbidden_features]

# Subset III: Generated features reduced (data_gf)
III_features = [col for col in data_gf.columns if col not in drop_comorb_features + forbidden_features]

In [ ]:
# Subset IV: Chi-square selected features (data_gf)
ordinal_features_significant = select_significants(data_gf, features['ordinal'], 'Target_3')
non_ordinal_features_significant = select_significants(data_gf, features['nominal'], 'Target_3')

selected_features = ordinal_features_significant + non_ordinal_features_significant + features['num'] + features['generated']
IV_features = [col for col in selected_features if col not in drop_comorb_features]

In [ ]:
# Subset V: Image-related features (data)
image_features = [col for col in data.columns if 'CT' in col]

#### Hold-out test dataset and selection of feature subsets

In [ ]:
target_X_s, target_X_s_test, target_y_s, target_y_s_test, target_split_indices = {}, {}, {}, {}, {}

In [ ]:
for target in [0, 2, 3]:
    tmp_X = data.copy()
    tmp_y = tmp_X.pop(f'Target_{target}')
    tmp_X_train, tmp_X_test, tmp_y_train, tmp_y_test = train_test_split(tmp_X, tmp_y, test_size=0.3, random_state=random_state)
    
#     Use the following three lines if you haven't already created lists of train/val and test indices
    target_split_indices[target] = {'train/val': tmp_X_train.index.tolist(), 'test': tmp_X_test.index.tolist()}
    with open(join('data', 'indices', f'split_indices_target_{target}.json'), 'w') as fp:
        json.dump(target_split_indices[target], fp)
        
    # Use the following two lines if you've already created lists of train/val and test indices
#     with open(join('data', 'indices', f'split_indices_target_{target}.json'), 'r') as f:
#         target_split_indices[target] = json.load(f)

    # Create train/val dataset
    tmp_data = data.iloc[target_split_indices[target]['train/val']].reset_index(drop=True)
    tmp_data_gf = data_gf.iloc[target_split_indices[target]['train/val']].reset_index(drop=True)
    
    X_I = tmp_data[[col for col in tmp_data.columns if col not in forbidden_features]]
    X_II = tmp_data_gf[[col for col in tmp_data_gf.columns if col not in forbidden_features]]
    X_III = tmp_data_gf[[col for col in tmp_data_gf.columns if col not in drop_comorb_features + forbidden_features]]
    X_IV = tmp_data_gf[selected_features]
    X_V = tmp_data[image_features]
    target_X_s[target] = [X_I, X_II, X_III, X_IV, X_V]
    
    target_y_s[target] = tmp_data[f'Target_{target}']
    
    # Create test dataset
    tmp_test_data = data.iloc[target_split_indices[target]['test']].reset_index(drop=True)
    tmp_test_data_gf = data_gf.iloc[target_split_indices[target]['test']].reset_index(drop=True)
    
    X_I_test = tmp_test_data[[col for col in tmp_test_data.columns if col not in forbidden_features]]
    X_II_test = tmp_test_data_gf[[col for col in tmp_test_data_gf.columns if col not in forbidden_features]]
    X_III_test = tmp_test_data_gf[[col for col in tmp_test_data_gf.columns if col not in drop_comorb_features + forbidden_features]]
    X_IV_test = tmp_test_data_gf[selected_features]
    X_V_test = tmp_test_data[image_features]
    target_X_s_test[target] = [X_I_test, X_II_test, X_III_test, X_IV_test, X_V_test]
    
    target_y_s_test[target] = tmp_test_data[f'Target_{target}']

### Target 0: Höchster Behandlungsstatus

In [ ]:
target = 0
X_s = target_X_s[target]
y = target_y_s[0].copy()

In [ ]:
y.value_counts(normalize=True)

#### Results with combination of classes but no upsampling
Combine class 0,1 and 2 into one class. Classes now:
* 0 -> Nicht beantwortet, Ambulant, Notaufnahme
* 1 -> Stationär
* 2 -> IMC
* 3 -> ICU

In [ ]:
y_replaced = y.replace({1:0, 2:0, 3:1, 4:2, 5:3})

In [ ]:
y_replaced.value_counts(normalize=True)

In [ ]:
d_results = {
    'feature_subgroup': [],
    'model': [],
    'reference': [],
    'accuracy': [],
    'PPV': [],
    'sensitivity': [],
    'f1_score': [],
    'p-value': []
}

In [ ]:
best_model, best_model_type, best_i, best_X_train, best_y_train, best_X_test, best_y_test, best_acc = None, None, None, None, None, None, None, 0

In [ ]:
for i, X in enumerate(X_s):
    print(f'Feature subset {i+1}')
    X_train, X_test, y_train, y_test = train_test_split(X, y_replaced, test_size=0.3, random_state=random_state)
    
    for model_type in ['Random forest', 'Gradient boosting']:
        print(model_type)
        model = RandomForestClassifier(random_state=random_state) if model_type == 'Random forest' else GradientBoostingClassifier(random_state=random_state)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # Perform McNemar significance test
        majority_vote = np.array([y_test.mode()[0]] * len(y_test))
        reference = accuracy_score(y_test, majority_vote)
        d_results['reference'].append(reference)
        mcnemar_result = mcnemar(table=[[sum(y_test & predictions), sum(~y_test & predictions)],
                                      [sum(y_test & ~majority_vote), sum(~y_test & ~majority_vote)]], correction=True)
        p_value = mcnemar_result.pvalue
        d_results['p-value'].append(p_value)
        
        d_results['model'].append(model_type)

        acc = accuracy_score(y_test, predictions)
        d_results['feature_subgroup'].append(i+1)
        d_results['accuracy'].append(acc)
        d_results['PPV'].append(precision_score(y_test, predictions, average='weighted'))
        d_results['sensitivity'].append(recall_score(y_test, predictions, average='weighted'))
        d_results['f1_score'].append(f1_score(y_test, predictions, average='weighted'))
        print(f'Accuracy: {acc}')
        print(classification_report(y_test, predictions))
        if acc > best_acc:
            best_acc = acc
            best_i = i+1
            best_X_train = X_train
            best_y_train = y_train
            best_X_test = X_test
            best_y_test = y_test
            best_model = model
            best_model_type = model_type

In [ ]:
print(f'Best accuracy: {best_acc}\nBest model type: {best_model_type}\nBest feature subset: {best_i}')

In [ ]:
results = pd.DataFrame(d_results)

d_results_restructured = {
    'feature_subset': [],
    'accuracy': [],
    'sensitivity': [],
    'PPV': []
}

for f in [1,2,3,4,5,6]:
    d_results_restructured['feature_subset'].append(f)
    for metric in ['accuracy', 'sensitivity', 'PPV']:
        rf = results[(results['feature_subgroup'] == f) & (results['model'] == 'Random forest')][metric].reset_index(drop=True).iloc[0]
        gb = results[(results['feature_subgroup'] == f) & (results['model'] == 'Gradient boosting')][metric].reset_index(drop=True).iloc[0]
        m_str = f'{gb*100:.2f}% ({rf*100:.2f}%)'
        d_results_restructured[metric].append(m_str)

pd.DataFrame(d_results_restructured)

In [ ]:
results

In [ ]:
feature_names = list(best_X_train.columns)
importances = best_model.feature_importances_

# Sort importances and get the indices of the 10 most important features
sorted_idx = np.argsort(importances)[-10:]

# Create a bar plot of the 10 most important feature importances
plt.barh(range(10), importances[sorted_idx], align='center')
plt.yticks(range(10), [feature_names[i] for i in sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Feature Importances of the 10 most important features')
plt.show()

#### Hyperparameter optimization

In [ ]:
if best_model_type == 'Random forest':
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [5, 10, 15, 20, 25, 30, None],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
        'max_features': ['auto', 'sqrt', 'log2', None],
        'random_state': [random_state]
    }
    
    model = RandomForestClassifier()

else:
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'learning_rate': [0.1, 0.05, 0.01],
        'max_depth': [1, 3, 5, 7, 9],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
        'subsample': [0.1, 0.5, 1.0],
        'max_features': ['auto', 'sqrt', 'log2', None],
        'loss': ['deviance', 'exponential']
    }
    
    model = GradientBoostingClassifier()

In [ ]:
# Create an instance of the RandomizedSearchCV
if perform_ho:
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=n_iter, cv=cv, random_state=random_state, verbose=verbose)

In [ ]:
%%time
# Fit the RandomizedSearchCV to the data
if perform_ho:
    random_search.fit(best_X_train, best_y_train)

In [ ]:
if perform_ho:
    # Print the best score
    print("Best Score: ",random_search.best_score_)

    # Print the best parameters
    print("Best Parameters: ",random_search.best_params_)

##### Evaluate final model on validation data

In [ ]:
if perform_ho:
    # Train model on training data
    model = RandomForestClassifier(**random_search.best_params_) if best_model_type == 'Random forest' else GradientBoostingClassifier(**random_search.best_params_)
    model.fit(best_X_train, best_y_train)
    predictions = model.predict(best_X_test)
    
    # Evaluate model on validation data
    print(f'Accuracy: {accuracy_score(best_y_test, predictions)}')
    print(f"PPV (precision): {precision_score(best_y_test, predictions, average='weighted')}")
    print(f"Sensitivity (recall): {recall_score(best_y_test, predictions, average='weighted')}")
    print(f"F1 score: {f1_score(best_y_test, predictions, average='weighted')}")

    print(classification_report(best_y_test, predictions))

##### Evaluate final model on hold-out test data (trained on train and validation data)

In [ ]:
if perform_ho:
    # Train model on training+validation data
    final_model = RandomForestClassifier(**random_search.best_params_) if best_model_type == 'Random forest' else GradientBoostingClassifier(**random_search.best_params_)
    final_model.fit(target_X_s[target][best_i-1], target_y_s[target])
    predictions = final_model.predict(target_X_s_test[target][best_i-1])
    
    # Evaluate model on hold-out test data
    print(f'Accuracy: {accuracy_score(target_y_s_test[target], predictions)}')
    print(f"PPV (precision): {precision_score(target_y_s_test[target], predictions, average='weighted')}")
    print(f"Sensitivity (recall): {recall_score(target_y_s_test[target], predictions, average='weighted')}")
    print(f"F1 score: {f1_score(target_y_s_test[target], predictions, average='weighted')}")

    print(classification_report(target_y_s_test[target], predictions))

##### Plot feature importances of final model

In [ ]:
if perform_ho:
    feature_names = list(best_X_train.columns)
    importances = final_model.feature_importances_

    # Sort importances and get the indices of the 10 most important features
    sorted_idx = np.argsort(importances)[-10:]

    # Create a bar plot of the 10 most important feature importances
    plt.barh(range(10), importances[sorted_idx], align='center')
    plt.yticks(range(10), [feature_names[i] for i in sorted_idx])
    plt.xlabel('Feature Importance')
    plt.title('Feature Importances of the 10 most important features')
    plt.show()

### Target 2: Höchster Beatmungsbedarf

In [ ]:
target = 2
X_s = target_X_s[target]
y = target_y_s[target].copy()

In [ ]:
y.value_counts(normalize=True)

In [ ]:
d_results = {
    'feature_subgroup': [],
    'model': [],
    'reference': [],
    'accuracy': [],
    'PPV': [],
    'sensitivity': [],
    'f1_score': [],
    'p-value': []
}

In [ ]:
best_model, best_model_type, best_i, best_X_train, best_y_train, best_X_test, best_y_test, best_acc = None, None, None, None, None, None, None, 0

In [ ]:
for i, X in enumerate(X_s):
    print(f'Feature subset {i+1}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    
    for model_type in ['Random forest', 'Gradient boosting']:
        print(model_type)
        model = RandomForestClassifier(random_state=random_state) if model_type == 'Random forest' else GradientBoostingClassifier(random_state=random_state)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # Perform McNemar significance test
        majority_vote = np.array([y_test.mode()[0]] * len(y_test))
        reference = accuracy_score(y_test, majority_vote)
        d_results['reference'].append(reference)
        mcnemar_result = mcnemar(table=[[sum(y_test & predictions), sum(~y_test & predictions)],
                                      [sum(y_test & ~majority_vote), sum(~y_test & ~majority_vote)]], correction=True)
        p_value = mcnemar_result.pvalue
        d_results['p-value'].append(p_value)
        
        d_results['model'].append(model_type)

        acc = accuracy_score(y_test, predictions)
        d_results['feature_subgroup'].append(i+1)
        d_results['accuracy'].append(acc)
        d_results['PPV'].append(precision_score(y_test, predictions, average='weighted'))
        d_results['sensitivity'].append(recall_score(y_test, predictions, average='weighted'))
        d_results['f1_score'].append(f1_score(y_test, predictions, average='weighted'))
        print(f'Accuracy: {acc}')
        print(classification_report(y_test, predictions))
        if acc > best_acc:
            best_acc = acc
            best_i = i+1
            best_X_train = X_train
            best_y_train = y_train
            best_X_test = X_test
            best_y_test = y_test
            best_model = model
            best_model_type = model_type

In [ ]:
print(f'Best accuracy: {best_acc}\nBest model type: {best_model_type}\nBest feature subset: {best_i}')

In [ ]:
results = pd.DataFrame(d_results)

d_results_restructured = {
    'feature_subset': [],
    'accuracy': [],
    'sensitivity': [],
    'PPV': []
}

for f in [1,2,3,4,5,6]:
    d_results_restructured['feature_subset'].append(f)
    for metric in ['accuracy', 'sensitivity', 'PPV']:
        rf = results[(results['feature_subgroup'] == f) & (results['model'] == 'Random forest')][metric].reset_index(drop=True).iloc[0]
        gb = results[(results['feature_subgroup'] == f) & (results['model'] == 'Gradient boosting')][metric].reset_index(drop=True).iloc[0]
        m_str = f'{gb*100:.2f}% ({rf*100:.2f}%)'
        d_results_restructured[metric].append(m_str)

pd.DataFrame(d_results_restructured)

In [ ]:
results

In [ ]:
feature_names = list(best_X_train.columns)
importances = best_model.feature_importances_

# Sort importances and get the indices of the 10 most important features
sorted_idx = np.argsort(importances)[-10:]

# Create a bar plot of the 10 most important feature importances
plt.barh(range(10), importances[sorted_idx], align='center')
plt.yticks(range(10), [feature_names[i] for i in sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Feature Importances of the 10 most important features')
plt.show()

#### Hyperparameter optimization

In [ ]:
if best_model_type == 'Random forest':
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [5, 10, 15, 20, 25, 30, None],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
        'max_features': ['auto', 'sqrt', 'log2', None],
        'random_state': [random_state]
    }
    
    model = RandomForestClassifier()

else:
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'learning_rate': [0.1, 0.05, 0.01],
        'max_depth': [1, 3, 5, 7, 9],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
        'subsample': [0.1, 0.5, 1.0],
        'max_features': ['auto', 'sqrt', 'log2', None],
        'loss': ['deviance', 'exponential']
    }
    
    model = GradientBoostingClassifier()

In [ ]:
# Create an instance of the RandomizedSearchCV
if perform_ho:
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=n_iter, cv=cv, random_state=random_state, verbose=verbose)

In [ ]:
%%time
# Fit the RandomizedSearchCV to the data
if perform_ho:
    random_search.fit(best_X_train, best_y_train)

In [ ]:
if perform_ho:
    # Print the best score
    print("Best Score: ",random_search.best_score_)

    # Print the best parameters
    print("Best Parameters: ",random_search.best_params_)

##### Evaluate final model on validation data

In [ ]:
if perform_ho:
    # Train model on training data
    model = RandomForestClassifier(**random_search.best_params_) if best_model_type == 'Random forest' else GradientBoostingClassifier(**random_search.best_params_)
    model.fit(best_X_train, best_y_train)
    predictions = model.predict(best_X_test)
    
    # Evaluate model on validation data
    print(f'Accuracy: {accuracy_score(best_y_test, predictions)}')
    print(f"PPV (precision): {precision_score(best_y_test, predictions, average='weighted')}")
    print(f"Sensitivity (recall): {recall_score(best_y_test, predictions, average='weighted')}")
    print(f"F1 score: {f1_score(best_y_test, predictions, average='weighted')}")

    print(classification_report(best_y_test, predictions))

##### Evaluate final model on hold-out test data (trained on train and validation data)

In [ ]:
if perform_ho:
    # Train model on training+validation data
    final_model = RandomForestClassifier(**random_search.best_params_) if best_model_type == 'Random forest' else GradientBoostingClassifier(**random_search.best_params_)
    final_model.fit(target_X_s[target][best_i-1], target_y_s[target])
    predictions = final_model.predict(target_X_s_test[target][best_i-1])
    
    # Evaluate model on hold-out test data
    print(f'Accuracy: {accuracy_score(target_y_s_test[target], predictions)}')
    print(f"PPV (precision): {precision_score(target_y_s_test[target], predictions, average='weighted')}")
    print(f"Sensitivity (recall): {recall_score(target_y_s_test[target], predictions, average='weighted')}")
    print(f"F1 score: {f1_score(target_y_s_test[target], predictions, average='weighted')}")

    print(classification_report(target_y_s_test[target], predictions))

##### Plot feature importances of final model

In [ ]:
if perform_ho:
    feature_names = list(best_X_train.columns)
    importances = final_model.feature_importances_

    # Sort importances and get the indices of the 10 most important features
    sorted_idx = np.argsort(importances)[-10:]

    # Create a bar plot of the 10 most important feature importances
    plt.barh(range(10), importances[sorted_idx], align='center')
    plt.yticks(range(10), [feature_names[i] for i in sorted_idx])
    plt.xlabel('Feature Importance')
    plt.title('Feature Importances of the 10 most important features')
    plt.show()

### Target 3: Binärer Score

In [ ]:
target = 3
X_s = target_X_s[target]
y = target_y_s[target].copy()

In [ ]:
y.value_counts(normalize=True)

In [ ]:
d_results = {
    'feature_subgroup': [],
    'model': [],
    'reference': [],
    'accuracy': [],
    'PPV': [],
    'sensitivity': [],
    'f1_score': [],
    'p-value': []
}

In [ ]:
best_model, best_model_type, best_i, best_X_train, best_y_train, best_X_test, best_y_test, best_acc = None, None, None, None, None, None, None, 0

In [ ]:
for i, X in enumerate(X_s):
    print(f'Feature subset {i+1}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
    
    for model_type in ['Random forest', 'Gradient boosting']:
        print(model_type)
        model = RandomForestClassifier(random_state=random_state) if model_type == 'Random forest' else GradientBoostingClassifier(random_state=random_state)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)

        # Perform McNemar significance test
        majority_vote = np.array([y_test.mode()[0]] * len(y_test))
        reference = accuracy_score(y_test, majority_vote)
        d_results['reference'].append(reference)
        mcnemar_result = mcnemar(table=[[sum(y_test & predictions), sum(~y_test & predictions)],
                                      [sum(y_test & ~majority_vote), sum(~y_test & ~majority_vote)]], correction=True)
        p_value = mcnemar_result.pvalue
        d_results['p-value'].append(p_value)
        
        d_results['model'].append(model_type)

        acc = accuracy_score(y_test, predictions)
        d_results['feature_subgroup'].append(i+1)
        d_results['accuracy'].append(acc)
        d_results['PPV'].append(precision_score(y_test, predictions, average='weighted'))
        d_results['sensitivity'].append(recall_score(y_test, predictions, average='weighted'))
        d_results['f1_score'].append(f1_score(y_test, predictions, average='weighted'))
        print(f'Accuracy: {acc}')
        print(classification_report(y_test, predictions))
        if acc > best_acc:
            best_acc = acc
            best_i = i+1
            best_X_train = X_train
            best_y_train = y_train
            best_X_test = X_test
            best_y_test = y_test
            best_model = model
            best_model_type = model_type

In [ ]:
print(f'Best accuracy: {best_acc}\nBest model type: {best_model_type}\nBest feature subset: {best_i}')

In [ ]:
results = pd.DataFrame(d_results)

d_results_restructured = {
    'feature_subset': [],
    'accuracy': [],
    'sensitivity': [],
    'PPV': []
}

for f in [1,2,3,4,5,6]:
    d_results_restructured['feature_subset'].append(f)
    for metric in ['accuracy', 'sensitivity', 'PPV']:
        rf = results[(results['feature_subgroup'] == f) & (results['model'] == 'Random forest')][metric].reset_index(drop=True).iloc[0]
        gb = results[(results['feature_subgroup'] == f) & (results['model'] == 'Gradient boosting')][metric].reset_index(drop=True).iloc[0]
        m_str = f'{gb*100:.2f}% ({rf*100:.2f}%)'
        d_results_restructured[metric].append(m_str)

pd.DataFrame(d_results_restructured)

In [ ]:
results

In [ ]:
feature_names = list(best_X_train.columns)
importances = best_model.feature_importances_

# Sort importances and get the indices of the 10 most important features
sorted_idx = np.argsort(importances)[-10:]

# Create a bar plot of the 10 most important feature importances
plt.barh(range(10), importances[sorted_idx], align='center')
plt.yticks(range(10), [feature_names[i] for i in sorted_idx])
plt.xlabel('Feature Importance')
plt.title('Feature Importances of the 10 most important features')
plt.show()

#### Hyperparameter optimization

In [ ]:
if best_model_type == 'Random forest':
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [5, 10, 15, 20, 25, 30, None],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
        'max_features': ['auto', 'sqrt', 'log2', None],
        'random_state': [random_state]
    }
    
    model = RandomForestClassifier()

else:
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500],
        'learning_rate': [0.1, 0.05, 0.01],
        'max_depth': [1, 3, 5, 7, 9],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 4, 8],
        'subsample': [0.1, 0.5, 1.0],
        'max_features': ['auto', 'sqrt', 'log2', None],
        'loss': ['deviance', 'exponential']
    }
    
    model = GradientBoostingClassifier()

In [ ]:
# Create an instance of the RandomizedSearchCV
if perform_ho:
    random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=n_iter, cv=cv, random_state=random_state, verbose=verbose)

In [ ]:
%%time
# Fit the RandomizedSearchCV to the data
if perform_ho:
    random_search.fit(best_X_train, best_y_train)

In [ ]:
if perform_ho:
    # Print the best score
    print("Best Score: ",random_search.best_score_)

    # Print the best parameters
    print("Best Parameters: ",random_search.best_params_)

##### Evaluate final model on validation data

In [ ]:
if perform_ho:
    # Train model on training data
    model = RandomForestClassifier(**random_search.best_params_) if best_model_type == 'Random forest' else GradientBoostingClassifier(**random_search.best_params_)
    model.fit(best_X_train, best_y_train)
    predictions = model.predict(best_X_test)
    
    # Evaluate model on validation data
    print(f'Accuracy: {accuracy_score(best_y_test, predictions)}')
    print(f"PPV (precision): {precision_score(best_y_test, predictions, average='weighted')}")
    print(f"Sensitivity (recall): {recall_score(best_y_test, predictions, average='weighted')}")
    print(f"F1 score: {f1_score(best_y_test, predictions, average='weighted')}")

    print(classification_report(best_y_test, predictions))

##### Evaluate final model on hold-out test data (trained on train and validation data)

In [ ]:
if perform_ho:
    # Train model on training+validation data
    final_model = RandomForestClassifier(**random_search.best_params_) if best_model_type == 'Random forest' else GradientBoostingClassifier(**random_search.best_params_)
    final_model.fit(target_X_s[target][best_i-1], target_y_s[target])
    predictions = final_model.predict(target_X_s_test[target][best_i-1])
    
    # Evaluate model on hold-out test data
    print(f'Accuracy: {accuracy_score(target_y_s_test[target], predictions)}')
    print(f"PPV (precision): {precision_score(target_y_s_test[target], predictions, average='weighted')}")
    print(f"Sensitivity (recall): {recall_score(target_y_s_test[target], predictions, average='weighted')}")
    print(f"F1 score: {f1_score(target_y_s_test[target], predictions, average='weighted')}")

    print(classification_report(target_y_s_test[target], predictions))

##### Plot feature importances of final model

In [ ]:
if perform_ho:
    feature_names = list(best_X_train.columns)
    importances = final_model.feature_importances_

    # Sort importances and get the indices of the 10 most important features
    sorted_idx = np.argsort(importances)[-10:]

    # Create a bar plot of the 10 most important feature importances
    plt.barh(range(10), importances[sorted_idx], align='center')
    plt.yticks(range(10), [feature_names[i] for i in sorted_idx])
    plt.xlabel('Feature Importance')
    plt.title('Feature Importances of the 10 most important features')
    plt.show()

##### Location-based accuracy

In [ ]:
location_count = raw_data.iloc[target_split_indices[3]['test']]['Location'].value_counts().to_frame().reset_index(drop=False)
locations = [loc for loc in location_count['index'].unique() if location_count[location_count['index'] == loc]['Location'].reset_index(drop=True).iloc[0] >= 5]

In [ ]:
target_X_s_test_loc = target_X_s_test[target][best_i-1]
target_X_s_test_loc['Location'] = list(raw_data.iloc[target_split_indices[target]['test']]['Location'])

In [ ]:
target_y_s_test_loc = pd.DataFrame({'y': target_y_s_test[target], 'location': list(raw_data.iloc[target_split_indices[target]['test']]['Location'])})

In [ ]:
loc_accuracies = []
for loc in locations:
    tmp_loc_data_X = target_X_s_test_loc[target_X_s_test_loc['Location'] == loc]
    tmp_loc_data_y = target_y_s_test_loc[target_X_s_test_loc['Location'] == loc]
    tmp_loc_data_X.drop('Location', inplace=True, axis=1)
    
    loc_predictions = final_model.predict(tmp_loc_data_X)
    loc_acc = accuracy_score(tmp_loc_data_y['y'], loc_predictions)
    loc_accuracies.append(loc_acc)

In [ ]:
print(f'Location-specific results STD: {np.array(loc_accuracies).std()}')